# Create latest summary table which contain all the financial metrics for stock from 2022 to current

In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('Share.csv')
df

,Unnamed: 0,stock_code,pcf,div_5year_growth
0,0,5250,9.620,3.211
1,1,0800EA,36.661,0.000
2,2,7167,8.926,27.226
3,3,7086,122.935,0.000
4,4,5198,7.515,0.000
...,...,...,...,...
983,983,2283,13.774,0.000
984,984,0094,3.792,0.000
985,985,5131,18.297,14.870
986,986,7016,5.352,-0.448


In [3]:
del df['Unnamed: 0']

In [4]:
df[df['stock_code']=='03006']

,stock_code,pcf,div_5year_growth
600,03006,50.013,0.000


In [5]:
df2= pd.read_csv('pepb.csv')

In [6]:
df2

,Unnamed: 0,stock_code,pe,pb
0,0,5250,23.98,27.414
1,1,0800EA,-,0.276
2,2,7167,13.42,1.153
3,3,7086,-,0.831
4,4,5198,22.34,0.429
...,...,...,...,...
983,983,1155,12.72,1.228
984,984,5022,-,0.575
985,985,7099,19.12,0.360
986,986,5299,19.87,0.550


In [7]:
del df2['Unnamed: 0']

In [8]:
df2[df2['stock_code']=='03006']

,stock_code,pe,pb
596,03006,-,3.109


In [9]:
df3=pd.merge(df,df2,on='stock_code',how='left')

In [10]:
df3

,stock_code,pcf,div_5year_growth,pe,pb
0,5250,9.620,3.211,23.98,27.414
1,0800EA,36.661,0.000,-,0.276
2,7167,8.926,27.226,13.42,1.153
3,7086,122.935,0.000,-,0.831
4,5198,7.515,0.000,22.34,0.429
...,...,...,...,...,...
983,2283,13.774,0.000,4.13,0.680
984,0094,3.792,0.000,-,2.161
985,5131,18.297,14.870,21.97,1.668
986,7016,5.352,-0.448,16.66,0.255


In [11]:
df3[df3['stock_code']=='03006']

,stock_code,pcf,div_5year_growth,pe,pb
600,03006,50.013,0.000,-,3.109


In [12]:
df3.dtypes

stock_code          object
pcf                 object
div_5year_growth    object
pe                  object
pb                  object
dtype: object

### Replace dash(-) with 0 

In [13]:
df3.loc[df3['pcf'] == '-']

,stock_code,pcf,div_5year_growth,pe,pb
433,5280,-,-,-,-
529,5090,-,-,-,-
614,5298,-,-,-,-


In [14]:
# replace dash with 0
df3 = df3.replace('-', 0)

In [15]:
df3[df3['stock_code']=='5298']

,stock_code,pcf,div_5year_growth,pe,pb
614,5298,0,0,0,0


### Change data types

In [16]:
df3.dtypes

stock_code          object
pcf                 object
div_5year_growth    object
pe                  object
pb                  object
dtype: object

In [17]:
# Replace the thousand separator ',' to ensure the whole column is in a consistent format
df3['pcf'] = df3['pcf'].str.replace(',', '')

In [18]:
df3['pcf']=df3['pcf'].astype(float).round(3)

In [19]:
df3['div_5year_growth']=df3['div_5year_growth'].astype(float).round(3)


In [20]:
# Replace the thousand separator ',' to ensure the whole column is in a consistent format
df3['pe'] = df3['pe'].str.replace(',', '')

In [21]:
df3['pe']=df3['pe'].astype(float).round(3)


In [22]:
df3['pb']=df3['pb'].astype(float).round(3)


In [23]:
df3.dtypes

stock_code           object
pcf                 float64
div_5year_growth    float64
pe                  float64
pb                  float64
dtype: object

In [24]:
df3

,stock_code,pcf,div_5year_growth,pe,pb
0,5250,9.620,3.211,23.98,27.414
1,0800EA,36.661,0.000,NaN,0.276
2,7167,8.926,27.226,13.42,1.153
3,7086,122.935,0.000,NaN,0.831
4,5198,7.515,0.000,22.34,0.429
...,...,...,...,...,...
983,2283,13.774,0.000,4.13,0.680
984,0094,3.792,0.000,NaN,2.161
985,5131,18.297,14.870,21.97,1.668
986,7016,5.352,-0.448,16.66,0.255


### Join with other financial metrics 

In [25]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import timedelta

In [26]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing_no2'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

### Use sql query to get the latest year data of the stock

In [27]:
# using sql query to get recent date
sql2 ='''SELECT to_char(MAX(to_date("Year",'YYYYMMDD')),'YYYY') AS "latest_year","Stock Code" 
FROM final_summ
GROUP BY "Stock Code"
ORDER BY "latest_year" ;'''
cursor.execute(sql2)
temp_table = cursor.fetchall()
#print(recent_date[0])
df = pd.DataFrame(temp_table, columns =['Latest_year', 'stock_code'])
conn.commit()
conn.close()

In [28]:
df

,Latest_year,stock_code
0,2018,5622
1,2018,7285
2,2018,8133
3,2018,0105
4,2018,0195
...,...,...
985,2023,5027
986,2023,6718
987,2023,6399
988,2023,7811


In [29]:
df3

,stock_code,pcf,div_5year_growth,pe,pb
0,5250,9.620,3.211,23.98,27.414
1,0800EA,36.661,0.000,NaN,0.276
2,7167,8.926,27.226,13.42,1.153
3,7086,122.935,0.000,NaN,0.831
4,5198,7.515,0.000,22.34,0.429
...,...,...,...,...,...
983,2283,13.774,0.000,4.13,0.680
984,0094,3.792,0.000,NaN,2.161
985,5131,18.297,14.870,21.97,1.668
986,7016,5.352,-0.448,16.66,0.255


In [30]:
df4=pd.merge(df,df3,on='stock_code',how='left')

In [31]:
df4

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb
0,2018,5622,0.376,0.000,58.18,1.292
1,2018,7285,10.197,-34.881,NaN,1.891
2,2018,8133,5.513,-3.581,NaN,1.233
3,2018,0105,6.790,0.000,NaN,0.641
4,2018,0195,10.074,0.000,20.90,0.859
...,...,...,...,...,...,...
985,2023,5027,8.692,14.870,NaN,2.177
986,2023,6718,11.000,-4.365,10.25,0.367
987,2023,6399,2.411,-11.595,9.58,2.779
988,2023,7811,7.394,-30.117,7.61,0.590


In [32]:
df4['Latest_year']=df4['Latest_year'].replace(['2018','2019','2020','2021'],'2022')

In [33]:
df4

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb
0,2022,5622,0.376,0.000,58.18,1.292
1,2022,7285,10.197,-34.881,NaN,1.891
2,2022,8133,5.513,-3.581,NaN,1.233
3,2022,0105,6.790,0.000,NaN,0.641
4,2022,0195,10.074,0.000,20.90,0.859
...,...,...,...,...,...,...
985,2023,5027,8.692,14.870,NaN,2.177
986,2023,6718,11.000,-4.365,10.25,0.367
987,2023,6399,2.411,-11.595,9.58,2.779
988,2023,7811,7.394,-30.117,7.61,0.590


In [34]:
df4['Latest_year'].unique()

array(['2022', '2023'], dtype=object)

In [35]:
df4[df4['Latest_year']=='2023']

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb
984,2023,7293,9.874,33.890,17.72,1.939
985,2023,5027,8.692,14.870,NaN,2.177
986,2023,6718,11.000,-4.365,10.25,0.367
987,2023,6399,2.411,-11.595,9.58,2.779
988,2023,7811,7.394,-30.117,7.61,0.590
989,2023,5302,39.910,0.000,34.86,5.614


In [36]:
df4

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb
0,2022,5622,0.376,0.000,58.18,1.292
1,2022,7285,10.197,-34.881,NaN,1.891
2,2022,8133,5.513,-3.581,NaN,1.233
3,2022,0105,6.790,0.000,NaN,0.641
4,2022,0195,10.074,0.000,20.90,0.859
...,...,...,...,...,...,...
985,2023,5027,8.692,14.870,NaN,2.177
986,2023,6718,11.000,-4.365,10.25,0.367
987,2023,6399,2.411,-11.595,9.58,2.779
988,2023,7811,7.394,-30.117,7.61,0.590


### Connect to database

In [37]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing_no2'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

In [38]:
# using sql query to get recent date
sql2 ='''SELECT "Stock Code", "Year", "Dividend Status", "Dividend Yield(%)", "Dividend(RM)", "Earning per Share(EPS)", "DPR(%)", "ROE(%)"
FROM public.final_summ;'''

cursor.execute(sql2)
summary_t = cursor.fetchall()
#print(recent_date[0])
summ = pd.DataFrame(summary_t, columns =['stock_code', 'Latest_year', 'div_status', "dividend_y_%", "div_dividend(RM)", 'quar_eps',"annual_dp(%)",  "quar_roe(%)"])
conn.commit()
conn.close()

In [39]:
summ

,stock_code,Latest_year,div_status,dividend_y_%,div_dividend(RM),quar_eps,annual_dp(%),quar_roe(%)
0,0001,2018,Y,1.650,0.0150,0.021,0.0,10.9
1,0001,2019,Y,1.159,0.0150,0.029,51.0,9.5
2,0001,2020,Y,0.971,0.0150,0.034,44.0,10.1
3,0001,2021,Y,1.343,0.0150,0.034,44.0,8.7
4,0001,2022,Y,0.282,0.0050,0.035,57.0,8.3
...,...,...,...,...,...,...,...,...
3951,9962,2020,Y,2.101,0.0050,0.026,19.0,2.6
3952,9962,2021,Y,3.362,0.0100,0.033,30.0,3.4
3953,9962,2022,Y,1.754,0.0050,0.024,21.0,2.4
3954,9997,2021,Y,3.716,0.0225,0.126,18.0,12.9


In [40]:
df8=pd.merge(df4,summ,on=['stock_code','Latest_year'],how='left')

In [41]:
df8

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb,div_status,dividend_y_%,div_dividend(RM),quar_eps,annual_dp(%),quar_roe(%)
0,2022,5622,0.376,0.000,58.18,1.292,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,7285,10.197,-34.881,NaN,1.891,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,8133,5.513,-3.581,NaN,1.233,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,0105,6.790,0.000,NaN,0.641,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,0195,10.074,0.000,20.90,0.859,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
985,2023,5027,8.692,14.870,NaN,2.177,Y,5.753,0.100,0.130,77.0,14.8
986,2023,6718,11.000,-4.365,10.25,0.367,Y,2.885,0.030,0.101,30.0,3.0
987,2023,6399,2.411,-11.595,9.58,2.779,Y,4.006,0.030,0.039,77.0,17.4
988,2023,7811,7.394,-30.117,7.61,0.590,Y,4.595,0.034,0.085,40.0,6.1


In [42]:
df8[df8['stock_code']=='0005']

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb,div_status,dividend_y_%,div_dividend(RM),quar_eps,annual_dp(%),quar_roe(%)
447,2022,0005,3.572,0.0,NaN,2.709,N,0.0,0.0,-0.056,0.0,NaN


In [43]:
df8[df8['Latest_year']=='2023']

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb,div_status,dividend_y_%,div_dividend(RM),quar_eps,annual_dp(%),quar_roe(%)
984,2023,7293,9.874,33.890,17.72,1.939,Y,0.429,0.010,0.167,6.0,6.8
985,2023,5027,8.692,14.870,NaN,2.177,Y,5.753,0.100,0.130,77.0,14.8
986,2023,6718,11.000,-4.365,10.25,0.367,Y,2.885,0.030,0.101,30.0,3.0
987,2023,6399,2.411,-11.595,9.58,2.779,Y,4.006,0.030,0.039,77.0,17.4
988,2023,7811,7.394,-30.117,7.61,0.590,Y,4.595,0.034,0.085,40.0,6.1
989,2023,5302,39.910,0.000,34.86,5.614,Y,1.136,0.020,0.068,29.0,11.3


In [44]:
df8['div_status'] = df8['div_status'].fillna('N')

In [45]:
df8['div_status'].unique()

array(['N', 'Y'], dtype=object)

In [46]:
df8

,Latest_year,stock_code,pcf,div_5year_growth,pe,pb,div_status,dividend_y_%,div_dividend(RM),quar_eps,annual_dp(%),quar_roe(%)
0,2022,5622,0.376,0.000,58.18,1.292,N,NaN,NaN,NaN,NaN,NaN
1,2022,7285,10.197,-34.881,NaN,1.891,N,NaN,NaN,NaN,NaN,NaN
2,2022,8133,5.513,-3.581,NaN,1.233,N,NaN,NaN,NaN,NaN,NaN
3,2022,0105,6.790,0.000,NaN,0.641,N,NaN,NaN,NaN,NaN,NaN
4,2022,0195,10.074,0.000,20.90,0.859,N,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
985,2023,5027,8.692,14.870,NaN,2.177,Y,5.753,0.100,0.130,77.0,14.8
986,2023,6718,11.000,-4.365,10.25,0.367,Y,2.885,0.030,0.101,30.0,3.0
987,2023,6399,2.411,-11.595,9.58,2.779,Y,4.006,0.030,0.039,77.0,17.4
988,2023,7811,7.394,-30.117,7.61,0.590,Y,4.595,0.034,0.085,40.0,6.1


In [47]:
df8.dtypes

Latest_year          object
stock_code           object
pcf                 float64
div_5year_growth    float64
pe                  float64
pb                  float64
div_status           object
dividend_y_%        float64
div_dividend(RM)    float64
quar_eps            float64
annual_dp(%)        float64
quar_roe(%)         float64
dtype: object

In [48]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing_no2'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()


df8.to_sql("latest_sum", con=conn, if_exists='replace',
          index=False,index_label=False)
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

# sql1 = '''select * from stock_price;'''
# cursor.execute(sql1)
# count=0
# for i in cursor.fetchall():
#     count+=1
#     print(i)

conn.commit()
conn.close()